# Rf 6 0 5_Profilell
Likelihood and minimization: working with the profile likelihood estimator



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:59 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooAddPdf.h"
#include "RooMinimizer.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Create model and dataset
 -----------------------------------------------

Observable

In [3]:
RooRealVar x("x", "x", -20, 20);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Model (intentional strong correlations)

In [4]:
RooRealVar mean("mean", "mean of g1 and g2", 0, -10, 10);
RooRealVar sigma_g1("sigma_g1", "width of g1", 3);
RooGaussian g1("g1", "g1", x, mean, sigma_g1);

RooRealVar sigma_g2("sigma_g2", "width of g2", 4, 3.0, 6.0);
RooGaussian g2("g2", "g2", x, mean, sigma_g2);

RooRealVar frac("frac", "frac", 0.5, 0.0, 1.0);
RooAddPdf model("model", "model", RooArgList(g1, g2), frac);

Generate 1000 events

In [5]:
RooDataSet *data = model.generate(x, 1000);

Construct plain likelihood
 ---------------------------------------------------

Construct unbinned likelihood

In [6]:
RooAbsReal *nll = model.createNLL(*data, NumCPU(2));

Minimize likelihood w.r.t all parameters before making plots

In [7]:
RooMinimizer(*nll).migrad();

 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 frac         5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 mean         0.00000e+00  2.00000e+00   -1.00000e+01  1.00000e+01
     3 sigma_g2     4.00000e+00  3.00000e-01    3.00000e+00  6.00000e+00
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1500           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
[#1] INFO:Eval -- RooAbsTestStatistic::initMPMode: started 2 remote server process.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-03
 FCN=2660.22 FROM MIGRAD    STATUS=INITIATE       10 CALLS          11 TOTA

Plot likelihood scan frac

In [8]:
RooPlot *frame1 = frac.frame(Bins(10), Range(0.01, 0.95), Title("LL and profileLL in frac"));
nll->plotOn(frame1, ShiftToZero());

[#1] INFO:Eval -- RooAbsTestStatistic::initMPMode: started 2 remote server process.


Plot likelihood scan in sigma_g2

In [9]:
RooPlot *frame2 = sigma_g2.frame(Bins(10), Range(3.3, 5.0), Title("LL and profileLL in sigma_g2"));
nll->plotOn(frame2, ShiftToZero());

[#1] INFO:Eval -- RooAbsTestStatistic::initMPMode: started 2 remote server process.


Construct profile likelihood in frac
 -----------------------------------------------------------------------

The profile likelihood estimator on nll for frac will minimize nll w.r.t
 all floating parameters except frac for each evaluation

In [10]:
RooAbsReal *pll_frac = nll->createProfile(frac);

Plot the profile likelihood in frac

In [11]:
pll_frac->plotOn(frame1, LineColor(kRed));

[#1] INFO:Minization -- RooProfileLL::evaluate(nll_model_modelData_Profile[frac]) Creating instance of MINUIT
[#1] INFO:Minization -- RooProfileLL::evaluate(nll_model_modelData_Profile[frac]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Eval -- RooAbsTestStatistic::initMPMode: started 2 remote server process.
[#1] INFO:Minization -- RooProfileLL::evaluate(nll_model_modelData_Profile[frac]) minimum found at (frac=0.623915)
..................................................................................

Adjust frame maximum for visual clarity

In [12]:
frame1->SetMinimum(0);
frame1->SetMaximum(3);

Construct profile likelihood in sigma_g2
 -------------------------------------------------------------------------------

The profile likelihood estimator on nll for sigma_g2 will minimize nll
 w.r.t all floating parameters except sigma_g2 for each evaluation

In [13]:
RooAbsReal *pll_sigmag2 = nll->createProfile(sigma_g2);

Plot the profile likelihood in sigma_g2

In [14]:
pll_sigmag2->plotOn(frame2, LineColor(kRed));


[#1] INFO:Minization -- RooProfileLL::evaluate(nll_model_modelData_Profile[sigma_g2]) Creating instance of MINUIT
[#1] INFO:Minization -- RooProfileLL::evaluate(nll_model_modelData_Profile[sigma_g2]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Eval -- RooAbsTestStatistic::initMPMode: started 2 remote server process.
[#1] INFO:Minization -- RooProfileLL::evaluate(nll_model_modelData_Profile[sigma_g2]) minimum found at (sigma_g2=4.11588)
....................................................................................

Adjust frame maximum for visual clarity

In [15]:
frame2->SetMinimum(0);
frame2->SetMaximum(3);

Make canvas and draw rooplots

In [16]:
TCanvas *c = new TCanvas("rf605_profilell", "rf605_profilell", 800, 400);
c->Divide(2);
c->cd(1);
gPad->SetLeftMargin(0.15);
frame1->GetYaxis()->SetTitleOffset(1.4);
frame1->Draw();
c->cd(2);
gPad->SetLeftMargin(0.15);
frame2->GetYaxis()->SetTitleOffset(1.4);
frame2->Draw();

delete pll_frac;
delete pll_sigmag2;
delete nll;

Draw all canvases 

In [17]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()